# Homework 10

* **Build the best model on the kaggle [Housing Data](https://www.kaggle.com/c/house-prices-advanced-regression-techniques) that you can !**

* **Use only Ridge or lasso or elastic net.**

* **Upload a screenshot of your leaderboard ranking**

**import libraries**

In [221]:
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV


from sklearn.model_selection import train_test_split

In [222]:
df_train = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")
y_train = df_train.SalePrice
X_train_unclean = df_train.drop(columns=["SalePrice"])
X_test_unclean = df_test

In [223]:
print(df_train.shape, df_test.shape)

(1460, 81) (1459, 80)


In [224]:
#get all categorical columns
categories_train = X_train_unclean.select_dtypes(include=['object'])

categories_test = X_test_unclean.select_dtypes(include=['object'])

In [225]:
print(categories_test.shape)

(1459, 43)


Alley: NA = "none"  
MasVnrType: NA = "none"  
BsmtQual: NA = "none"  
BsmtCond: NA = "none"  
BsmtExposure: NA = "none"  
BsmtFinType1: NA = "none"  
BsmtFinType2: NA = "none"  
Electrical: NA = "SBrkr" (weil häufigste)    
FireplaceQu: NA = "none"  
GarageType: NA = "none"  
GarageFinish: NA = "none"  
GarageQual: NA = "none"  
GarageCond: NA = "none"  
PoolQC: NA = "none"  
Fence: NA = "none"  
MiscFeature: NA = "none"  

In [226]:
# clean the training data
cats_train = categories_train.drop(columns = "Electrical").fillna("None")
elec_train = pd.DataFrame(categories.Electrical.fillna("Sbrkr"))
rest_train = X_train_unclean.select_dtypes(exclude=['object'])
X_train = pd.concat([cats_train, elec_train, rest_train], axis = 1) 

In [227]:
# clean the test data
cats_test = categories_test.drop(columns = "Electrical").fillna("None")
elec_test = pd.DataFrame(categories_test.Electrical.fillna("Sbrkr"))
rest_test = X_test_unclean.select_dtypes(exclude=['object'])
X_test = pd.concat([cats_test, elec_test, rest_test], axis = 1) 

In [228]:
print(X_test.shape)

(1459, 80)


In [229]:
print(elec.shape, rest.shape, cats.shape)
print(cats.isna().sum(), elec.isna().sum(), rest.isna().sum())

(1460, 1) (1460, 36) (1460, 42)
MSZoning         0
Street           0
Alley            0
LotShape         0
LandContour      0
Utilities        0
LotConfig        0
LandSlope        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
ExterQual        0
ExterCond        0
Foundation       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2     0
Heating          0
HeatingQC        0
CentralAir       0
KitchenQual      0
Functional       0
FireplaceQu      0
GarageType       0
GarageFinish     0
GarageQual       0
GarageCond       0
PavedDrive       0
PoolQC           0
Fence            0
MiscFeature      0
SaleType         0
SaleCondition    0
dtype: int64 Electrical    0
dtype: int64 MSSubClass         0
LotFrontage      259
LotArea            0
OverallQual        0
OverallCond        0
YearBuilt          0
Ye

In [230]:
# instantiate the encoder and imputer

enc = OneHotEncoder(handle_unknown='ignore')

imp_num = SimpleImputer(missing_values = np.nan)

# define the columns on which operation to perform

num_cols = make_column_selector(dtype_include='number')
cat_cols = make_column_selector(dtype_exclude='number')

In [231]:
#create preprocessing step
preprocessor = ColumnTransformer(transformers =[("num", imp, num_cols),
                                                ("cat", enc, cat_cols),
                                               ], 
                                 
                                 remainder = "drop")

#create classifier
enet = ElasticNet(tol = 1e-2, random_state = 42, normalize = True)

#Create pipeline of preprocessor + lasso
clf = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", enet)])

#Define lasso_parameters searched to optimize
elas_param = {"classifier__alpha":np.arange(0.01,1,0.01), "classifier__l1_ratio":np.arange(0.01,1,0.01), "preprocessor__num__strategy":["most_frequent","mean","median"]}

clf = GridSearchCV(clf, elas_param, cv=10)

#clf.fit(X_train, y_train)

GridSearchCV(cv=10,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         SimpleImputer(strategy='median'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001FF2E7B0370>),
                                                                        ('cat',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         <sklearn.compose._column_transformer.make_column_selector object at 0x000001FF2E0A1FA0>)...
       0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4 , 0.41, 0.42, 0.43, 0.44,
       0.45, 0.46, 0.47, 0.48, 0.49, 0.5 , 0.51, 0.52, 0.53, 0.54, 0.55,
       0.56, 0.57, 0.58, 0.59, 0.6 , 0.61, 0.62, 0.63, 0.64, 0.65, 0.6

In [232]:
alpha_opt = clf.best_params_['classifier__alpha']
l1_opt = clf.best_params_['classifier__l1_ratio']
strat_opt = clf.best_params_['preprocessor__num__strategy']

print(f"The optimal parameters for the elastic net are: {alpha_opt, l1_opt, strat_opt}")

The optimal parameters for the elastic net are: (0.05, 0.99, 'median')


In [233]:
# implementing the fit using the best parameter

# preprocessing instantiation
enc = OneHotEncoder(handle_unknown='ignore')

imp = SimpleImputer(missing_values = np.nan, strategy = strat_opt)

#create preprocessing step
preprocessor = ColumnTransformer(transformers =[("num", imp, num_cols),("cat", enc, cat_cols)], remainder = "drop")

#create classifier
enet_opt = ElasticNet(alpha = alpha_opt, l1_ratio=l1_opt ,tol = 1e-2, random_state = 42, normalize = True)

# creating the pipe 
pipe = Pipeline(steps=[("preprocessor", preprocessor), ("classifier", enet_opt)])

# calculating the fit
pipe.fit(X_train, y_train)

# calculating the prediction 
y_pred = pipe.predict(X_test)

In [234]:
pred_df = pd.DataFrame(y_pred, index=df_test.index, columns=["SalePrice"])
pred_df.index += 1461
pred_df.to_csv('kaggle-submission-houseprice.csv', header=True, index_label='Id')
print(pred_df)

          SalePrice
1461  113019.284058
1462  158446.670992
1463  185306.250794
1464  201183.925089
1465  196780.283603
...             ...
2915   77316.367122
2916   81327.966936
2917  170619.734951
2918  111596.296735
2919  219968.944023

[1459 rows x 1 columns]
